In [ ]:
from sdv import Metadata
import os

print(os.getcwd())
from test_20_tables import fetch_data_from_sqlite

metadata, tables = fetch_data_from_sqlite()
metadata = Metadata(metadata)
metadata

In [5]:
metadata.to_json("1.json")

In [ ]:
#from sdv.utils import display_tables

#display_tables(tables)
for name, table in tables.items():
    print(name, table.shape)
from test_20_tables import save_tables
save_tables(tables, "input.xlsx")

In [ ]:
from sdv.relational import HMA1
import sys
import warnings
warnings.filterwarnings('ignore')
#from tutorials.newhma import NewModel as HMA1
#from tutorials.redirector import WriteableRedirector

#with WriteableRedirector():
model = HMA1(metadata)    

In [ ]:
import logging
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')# Configure stream handler for the cells
chandler = logging.StreamHandler()
chandler.setLevel(logging.WARNING)
chandler.setFormatter(formatter)
logging.basicConfig(handlers=[logging.FileHandler('sdv.log'), chandler])
logging.log(0, "1223")

In [ ]:
print(11)
model.fit(tables)

3s

In [ ]:
new_data = model.sample()
new_data

8s，很快

In [ ]:
save_tables(new_data)

In [ ]:
for name, table in new_data.items():
    print(name, table.shape)

In [ ]:
model.save('my_model_10k.pkl')

In [2]:
from sdv.relational import HMA1
import warnings
import pickle
warnings.filterwarnings('ignore')
loaded: HMA1 = HMA1.load('my_model_10k.pkl')

In [ ]:
new_data = loaded.sample(not_finalized=True)
new_data.keys()

pickle.dump(new_data, open('my_sample_not_finalized.pkl', 'wb'))

In [6]:
model = loaded
data_not_finalized = model.sample(num_rows=100, not_finalized=True)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Student Childing:   0%|          | 0/3 [00:00<?, ?it/s]

Enrollment:   0%|          | 0/100 [00:00<?, ?it/s]

BookLoan:   0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

Lab Childing:   0%|          | 0/1 [00:00<?, ?it/s]

LabEquipment:   0%|          | 0/100 [00:00<?, ?it/s]

LabEquipment Childing:   0%|          | 0/1 [00:00<?, ?it/s]

EquipmentMaintenance:   0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
pickle.dump(data_not_finalized, open('data_not_finalized_1k.pkl', 'wb'))

In [3]:
data_not_finalized = pickle.load(open('data_not_finalized_1k.pkl', 'rb') )

In [2]:
data_not_finalized = pickle.load(open('my_sample_not_finalized.pkl', 'rb') )

In [7]:
model = loaded
model.finalize(data_not_finalized)

  0%|          | 0/20 [00:00<?, ?it/s]

Enrollment:   0%|          | 0/2 [00:00<?, ?it/s]

Enrollment course_id Likelihoods:   0%|          | 0/100 [00:00<?, ?it/s]

BookLoan:   0%|          | 0/2 [00:00<?, ?it/s]

BookLoan book_id Likelihoods:   0%|          | 0/100 [00:00<?, ?it/s]

...

Schedule course_id Likelihoods:   0%|          | 0/100 [00:00<?, ?it/s]

ProjectMember:   0%|          | 0/2 [00:00<?, ?it/s]

CourseTextbook:   0%|          | 0/2 [00:00<?, ?it/s]

CourseTextbook textbook_id Likelihoods:   0%|          | 0/100 [00:00<?, ?it/s]

{'Student':     student_id                name                      email date_of_birth  \
 0        10000      Brittany Nunez       nicole36@example.org    2000-08-18   
 1        10001  Christopher Kelley   matthewgross@example.net    1997-01-01   
 2        10002      Bryan Mckenzie     nicholas31@example.net    2004-01-23   
 3        10003     Miranda Miranda   meyersashley@example.org    2005-11-12   
 4        10004           Amy Olsen          epena@example.net    1997-03-20   
 ..         ...                 ...                        ...           ...   
 95       10095   Christopher Floyd  collinsdonald@example.org    2003-11-01   
 96       10096      Julie Gonzales  michelebryant@example.net    1996-06-07   
 97       10097        Scott Thomas    houseamanda@example.net    2003-04-07   
 98       10098     Sharon Whitaker       joseph04@example.org    2005-07-17   
 99       10099          Brian Leon      michael52@example.com    2004-01-23   
 
    major  
 0   5661  
 1 

In [5]:
data_single = model.sample('Schedule', num_rows=100, sample_children=False)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Schedule:   0%|          | 0/2 [00:00<?, ?it/s]

100


  0%|          | 0/100 [00:00<?, ?it/s]

Schedule course_id Likelihoods:   0%|          | 0/100 [00:00<?, ?it/s]

100


  0%|          | 0/100 [00:00<?, ?it/s]

Schedule professor_id Likelihoods:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
save_tables({"Schedule":data_single}, "output_single.xlsx")

In [ ]:
#—— 生成Schedule表，并生成关联外键表
data_subset = model.sample('Lab', num_rows=100, sample_children=True)

In [ ]:
save_tables(data_subset, "output_subset.xlsx")